In [1]:
using Pkg; Pkg.activate(".")

  Activating project at `c:\Users\anton\.julia\dev\AssigningSecondaryStructure`


In [81]:
using AssigningSecondaryStructure
import AssigningSecondaryStructure as ASS

using Backboner, NearestNeighbors
using LinearAlgebra

In [99]:
coords = reshape(Protein.readpdb("test/data/1ASS.pdb")[1].backbone.coords, 3, 3, :)
L = size(coords, 3)

Ca_pos = coords[:, 2, :]

pad_pos = fill(NaN, 3)
C_pos = coords[:, 3, :]
O_pos = [ASS.get_oxygen_positions(coords) pad_pos]
N_pos = coords[:, 1, :]
H_pos = [pad_pos ASS.get_hydrogen_positions(coords)]

kdtree = KDTree(Ca_pos, Euclidean())

for i in axes(coords, 3)[[4]]
    js, dists = knn(kdtree, view(Ca_pos, :, 1), 10, true)
    Hbonds = map(js) do j
        i <= j <= i+2 && return false
        r_ON = norm(O_pos[:, i] - N_pos[:, j])
        r_CH = norm(C_pos[:, i] - H_pos[:, j])
        r_OH = norm(O_pos[:, i] - H_pos[:, j])
        r_CN = norm(C_pos[:, i] - N_pos[:, j])
        E = ASS.Q1Q2 * ASS.F * (1/r_ON + 1/r_CH - 1/r_OH - 1/r_CN)
        return E < ASS.CUTOFF
    end
    display(Hbonds)
end

10-element Vector{Bool}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0